TODO:
    
Decide on how to create 2010 geom table from 2000.
- We can create new geoid indices on 2000 tables to create a massive join
- We can ingest entire 01-37 tables individually into RAM in python and interpolate in ram in place and write to
     new 2000 tables
- For either of the above we can decide to write all results into a single massive table, including new geoid column


In [ ]:
import array, csv, json, math, multiprocessing, numpy, os, random, re, shutil
import shapely, shapely.wkb, struct, subprocess, sys, tempfile, threading, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

set_default_psql_database('census2010')

Crosswalk:  download and document
---------------------------------

In [ ]:
download_file('http://users.pop.umn.edu/~jps/NHGIS_block2000_to_block2010.zip', 'capture/NHGIS_block2000_to_block2010.zip')

In [ ]:
unzip_file('capture/NHGIS_block2000_to_block2010.zip')

In [ ]:
!cat capture/NHGIS_block2000_to_block2010/readme.txt

In [ ]:
!head capture/NHGIS_block2000_to_block2010/crosswalk_block2000_block2010_v002.csv

In [ ]:
psql('SELECT * FROM crosswalk_block2000_block2010 '
    "WHERE geoid2010='010010201002007'")

Crosswalk columns are:
block_id_2000, block_id_2010, weight, parea
We'll tentatively ignore the last.

For a data column X, we want to compute:
x2k(block_id_2010) = sum(x2k(block_id_2000)*weight(block_id_2000, block_id_2010)) for all block_id_2000 overlapping block_id_2010



Create crosswalk_block2000_block2010 database table
------------------------------------

In [ ]:
psql('CREATE TABLE crosswalk_block2000_block2010 '
    '(geoid2000 varchar, '
    ' geoid2010 varchar, '
    ' weight real, '
    ' parea real)')

In [ ]:
full_path = os.path.abspath('capture/NHGIS_block2000_to_block2010/crosswalk_block2000_block2010_v002.csv')
psql("COPY crosswalk_block2000_block2010 "
     "FROM '%s' DELIMITER ',' CSV;" % full_path,
     database='census2010')

In [ ]:
psql("UPDATE crosswalk_block2000_block2010 SET "
     "geoid2000 = right(geoid2000, 15), "
     "geoid2010 = right(geoid2010, 15)",
     database='census2010')

In [ ]:
psql("CREATE INDEX ON crosswalk_block2000_block2010 (geoid2010)",
     database='census2010')

In [ ]:
psql("CREATE INDEX ON crosswalk_block2000_block2010 (geoid2000)",
     database='census2010')

In [ ]:
psql('\d crosswalk_block2000_block2010', database='census2010')


Create interpolated tables
--------------------------

In [ ]:
psql('\d census2010_block_idxs')

In [ ]:
# Old code for creating a new interpolated table
#DROP TABLE IF EXISTS {new_table};
#DROP TABLE IF EXISTS {tmp_table};
#CREATE TABLE {tmp_table} (blockidx2010, geoid2010, {columns})
#AS SELECT MIN(blockidx2010), geoid2010, {sums}
#FROM crosswalk_block2000_block2010
#JOIN {old_table} USING (geoid2000)
#JOIN census2010_block_idxs USING (geoid2010)
#GROUP BY geoid2010;
#CREATE UNIQUE INDEX ON {tmp_table} (blockidx2010);
#CREATE UNIQUE INDEX ON {tmp_table} (geoid2010);
#ALTER TABLE {tmp_table} RENAME TO {new_table};


# census2010_block2010
# census2000_block2010

def interpolate_2000_to_2010(old_table, column, force=False):
    # old_table: sf1_2000_block_p001
    # census_table_name: p001
    # SUM(weight * p001001)
    dataset = 'census2000_block2010'
    
    cache_dir = 'columncache'
    
    # dataset, e.g. census2010 or census2000_int2010 or LEHD 2011
    # table within dataset, e.g. P001 for census, HAC for LEHD
    # column within table
    # {cache_dir}/{dataset}/{census_table}-{column}.numpy
    
    dir_name = '{cache_dir}/{dataset}'.format(**locals())
    !mkdir -p $dir_name
    
    cache_filename = '{dir_name}/{column}.numpy'.format(**locals())
    if os.path.exists(cache_filename) and not force:
        sys.stdout.write('{cache_filename} already exists\n'.format(**locals()))
        return

    query = """
SELECT SUM(weight * {column})::REAL
FROM crosswalk_block2000_block2010
JOIN {old_table} USING (geoid2000)
GROUP BY geoid2010
ORDER BY geoid2010;
""".format(**locals())

    data = numpy.array([0] + [x[0] for x in query_psql(query)],
                       dtype=numpy.float32)
    
    numpy_atomic_save(cache_filename, data)

# interpolate_2000_to_2010('sf1_2000_block_p001', 'p001001')

In [ ]:
pool = SimpleThreadPoolExecutor(8)  # seems good for 64GB RAM earthserve2

for table in sorted(get_table_names('sf1_2000_block_%')):
    for column in get_census_column_names_from_view(table):
        pool.submit(interpolate_2000_to_2010, table, column)

pool.shutdown()
None

Sanity-checking first interpolated table
----------------------------------------
The 2000 census includes Puerto Rico (FIPS 72), but the crosswalk doesn't, so checking that the overall populations match requires filtering out Puerto Rico from the original census count.

In [ ]:
psql('\d sf1_2000_int2010_p001')
psql('SELECT * FROM sf1_2000_int2010_p001 LIMIT 5;')
psql('select SUM(p001001) from sf1_2000_int2010_p001;')
psql('select SUM(p001001) from sf1_2000_block_p001;')
psql("select SUM(p001001) from sf1_2000_block_p001 WHERE LEFT(geoid2000,2) != '72';")

In [ ]:
psql('SELECT MAX(p001001) FROM sf1_2000_int2010_p001')
psql('SELECT MAX(p001001) FROM sf1_2000_block_p001')
psql('SELECT MAX(p001001) FROM sf1_2010_block_p001')


Add geoid index to geo2000 table
--------------------------------

In [ ]:
#in hindsight, should have called this geoid2000
psql("ALTER TABLE geo2000 ADD COLUMN geoid CHARACTER(15)",
     database='census2010')

In [ ]:
psql("UPDATE geo2000 SET geoid ="
     " (state || county || tract || block)",
     database='census2010')

In [ ]:
psql("CREATE INDEX ON geo2000 (geoid)",
     database='census2010')

In [ ]:
psql("SELECT geoid FROM geo2000 WHERE sumlev='101' LIMIT 10", database='census2010')

In [ ]:
# Show some 2000 'geoids'
psql("SELECT state || county || tract || block FROM geo2000 WHERE sumlev='101' LIMIT 10", database='census2010')

File identification (FILEID),state/US abbreviation (STUSAB),summary levels (SUMLEV),and the
geographic component codes (GEOCOMP) are critical elements in identifying the geographic level
for each record. The STUSAB field identifies the highest level of geography for the file. In the case
of state file,it identifies the individual state. For SF 1 files,the following FILEID and STUSAB codes
are used:
SF 1 state and state equivalent files ‘uSF1’ ‘AL-WY’
SF 1 advance national file ‘uSF1A’ ‘US’
SF 1 final national file ‘uSF1F’ ‘US’